# Retrieve data from OPS

In this notebook, we will retrieve data from the OPS (Open Patent Services) API. The OPS API provides access to a wide range of patent data, including bibliographic information, legal status, and full-text documents.

## Requirements

Please make sure you are in the right environment. To install the required packages, run the following command:

```bash
poetry install --with dev
```

Finally, please make sure you configure the OPS API key in the `.env` file at the root of the repository. You can find the `.env.example` file in the root of the repository. Copy it to `.env` and fill it with your OPS API key.

In [1]:
import os

def mask_value(value, show_last=10):
    if value is None:
        return None
    return '*' * (len(value) - show_last) + value[-show_last:]

consumer_key = os.environ.get('CONSUMER_KEY')
consumer_secret_key = os.environ.get('CONSUMER_SECRET_KEY')
ops_api_url = os.environ.get('OPS_API_URL')

print(f"ConsumerKey: {mask_value(consumer_key)}")
print(f"ConsumerSecret: {mask_value(consumer_secret_key)}")
print(f"OpsApiUrl: {ops_api_url}")

ConsumerKey: *********SUMER_KEY>
ConsumerSecret: ****************ECRET_KEY>
OpsApiUrl: https://ops.epo.org/3.2


## Use the OPS API

In this second part, we will use the OPS API to retrieve data. The goal is to retrieve the description and claims of a patent. Then we want to automated the process to get all the patents published between two dates. The output will be in a first time in a JSON format. In a second time, we will use the data to create a PostgreSQL database.

### Retrieve the description and claims of a patent

In [17]:
import requests
import base64

def get_access_token(api_url: str, consumer_key: str, consumer_secret_key: str) -> str:
    """Get access token from Ops API.

    Args:
        api_url (str): The Ops API URL.
        consumer_key (str): The consumer key for authentication.
        consumer_secret_key (str): The consumer secret key for authentication.
    Returns:
        str: The access token.
    Raises:
        Exception: If the request fails or the access token is not found.
    """
    
    # Encode the consumer key and secret key in base64
    base_64_encoded = base64.b64encode(bytes(f"{consumer_key}:{consumer_secret_key}", 'utf-8')).decode('utf-8')
    
    
    url = f"{api_url}/auth/accesstoken"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': f'Basic {base_64_encoded}'
    }
    data = {
        'grant_type': 'client_credentials'
    }
    
    try:
        # Make the request to get the access token
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()  # Raise an error for bad responses
        
        # Extract the access token from the response
        access_token = response.json().get('access_token')
        if not access_token:
            raise ValueError("Access token not found in the response.")
        
        return access_token
    
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {e}")

access_token = get_access_token(ops_api_url, consumer_key, consumer_secret_key)
print(f"AccessToken: {mask_value(access_token)}")

AccessToken: ******************XE6bArOODl


In [20]:
def get_patent_description(api_url: str, access_token: str, type: str = "publication", format: str = "epodoc", number: str = "EP1000000") -> list[str]:
    """Get patent data from Ops API.

    Args:
        api_url (str): The Ops API URL.
        access_token (str): The access token for authentication.
        type (str): Reference type (application, priority, publication).
        format (str): The format of the patent data (docdb, epodoc).
        number (str): The patent number.
    Returns:
        list[str]: The patent data in the specified format.
    Raises:
        Exception: If the request fails or the patent data is not found.
    """
    
    url = f"{api_url}/rest-services/published-data/{type}/{format}/{number}/description"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Accept': 'application/json'
    }
    
    try:
        # Make the request to get the patent data
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        
        # Extract the patent data from the response
        patent_data = response.json()
        if not patent_data:
            raise ValueError("Patent data not found in the response.")
        
        # Extract only the description from the patent data
        description_data = patent_data.get("ops:world-patent-data", {}).get("ftxt:fulltext-documents", {}).get("ftxt:fulltext-document", {}).get("description", {}).get("p", [])
        
        # Ensure the description is a list of strings
        if isinstance(description_data, dict):
            description_data = [description_data]
        
        description = [p["$"] for p in description_data if "$" in p]
        
        if not description:
            raise ValueError("Description not found in the response.")
        
        return description
    
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {e}")
    except KeyError as e:
        raise Exception(f"Unexpected response structure: {e}")
    
patent_data = get_patent_description(ops_api_url, access_token, type="publication", format="docdb", number="WO2023028077")
print(f"PatentDescription: {patent_data[:5]}")

PatentDescription: ['SODIUM CHANNEL INHIBITORS AND METHODS OF DESIGNING SAME CLAIM OF PRIORITY [0001] This application claims the benefit of priority under 35 U.S.C. § 119(e) to U.S. provisional application serial no.63/236,594, filed on August 24, 2021, which application is incorporated herein by reference. BACKGROUND [0002] The present invention relates to organic compounds useful for therapy in a mammal, particularly a human, and in particular to inhibitors of sodium channel (e.g., NaV1.7) that are useful for treating sodium channel-mediated diseases or conditions, such as pain, as well as other diseases and conditions associated with the modulation of sodium channels. The invention further includes methods of designing organic compounds that inhibit the NaV1.7 channel based on atom-resolution structures thereof, such as obtained by cryogenic electron microscopy (“Cryo- EM”, or “cryoEM”). [0003] Voltage-gated sodium channels are transmembrane proteins that initiate action potentials

In [19]:
def get_patent_claims(api_url: str, access_token: str, type: str = "publication", format: str = "epodoc", number: str = "EP1000000") -> list[str]:
    """Get patent claims from Ops API.

    Args:
        api_url (str): The Ops API URL.
        access_token (str): The access token for authentication.
        type (str): Reference type (application, priority, publication).
        format (str): The format of the patent data (docdb, epodoc).
        number (str): The patent number.
    Returns:
        list[str]: A list of patent claims in the specified format.
    Raises:
        Exception: If the request fails or the patent claims are not found.
    """
    
    url = f"{api_url}/rest-services/published-data/{type}/{format}/{number}/claims"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Accept': 'application/json'
    }
    
    try:
        # Make the request to get the patent claims
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        
        # Extract the patent claims from the response
        patent_data = response.json()
        if not patent_data:
            raise ValueError("Patent data not found in the response.")
        
        # Extract only the claims from the patent data
        claims_data = patent_data.get("ops:world-patent-data", {}).get("ftxt:fulltext-documents", {}).get("ftxt:fulltext-document", {}).get("claims", {}).get("claim", {}).get("claim-text", [])
        
        # Ensure claims_data is a list
        if isinstance(claims_data, dict):  # Single claim case
            claims_data = [claims_data]
        
        # Extract the claim text
        claims_text = [claim.get("$", "") for claim in claims_data]
        
        if not claims_text:
            raise ValueError("No claims found in the response.")
        
        return claims_text
    
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {e}")
    except KeyError as e:
        raise Exception(f"Unexpected response structure: {e}")
    
patent_claims = get_patent_claims(ops_api_url, access_token, type="publication", format="docdb", number="EP4424670A1")
print(f"PatentClaims: {patent_claims}")

PatentClaims: ['1. A compound represented by Formula (I):\n<img class="EMIRef" id="a5ae8339-e0d5-408e-b0b7-82d57f9279f0-ib0030" />', 'or pharmaceutically acceptable salt or stereoisomer thereof, wherein:\nGis N or CH;\nL<1>, L<2> and L<3> each are independently selected from bonds or C1-C8 alkylene;\nR<1> is selected from -C(O)OR<4> or -OC(O)R<9>;\nR<2> is selected from -C(O)OR<5>, C3-C8 cycloalkyl, C1-C8 alkyl, -OC(O)R<10>, -C(O)NHC10-C20 alkyl or -NHC(O)C10-C20 alkyl;\nR<3> is selected from\n<img class="EMIRef" id="a5ae8339-e0d5-408e-b0b7-82d57f9279f0-ib0031" />\nR<4> is C10-C20 alkyl;\nR<5> is C10-C20 alkyl;\nR<6> is selected from -C(O)(CH2)1-8N(C1-C8 alkyl)C1-C8 alkyl, C1-C8 alkyl or -(CH2)1-8C(O)OR<11>;\nR<7> is -(CH2)1-8N(C1-C8 alkyl)C1-C8 alkyl;\nR<8> is -(CH2)1-8N(C1-C8 alkyl)C1-C8 alkyl;\nR<9> is C10-C20 alkyl;\nR<10> is C10-C20 alkyl;\nR<11> is C10-C20 alkyl; and\n* denotes a ligation site.', '2. The compound or pharmaceutically acceptable salt or stereoisomer thereof accordi

In [21]:
def get_patent_biblio(api_url: str, access_token: str, type: str = "publication", format: str = "epodoc", number: str = "EP1000000") -> dict:
    """Get patent bibliographic data from Ops API.

    Args:
        api_url (str): The Ops API URL.
        access_token (str): The access token for authentication.
        type (str): Reference type (application, priority, publication).
        format (str): The format of the patent data (docdb, epodoc).
        number (str): The patent number.
    Returns:
        dict: The patent bibliographic data in the specified format.
        The dictionary contains the patent number, title, and country code.
        The title is a dictionary with language codes as keys and title text as values.
        The country code is extracted from the first applicant name.
    Example:
        {
            "number": "EP1000000",
            "title": {
                "en": "Example Title in English",
                "fr": "Titre d'exemple en français"
            },
            "country": "EP"
        }
    Raises:
        Exception: If the request fails or the bibliographic data is not found.
    """
    
    url = f"{api_url}/rest-services/published-data/{type}/{format}/{number}/biblio"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Accept': 'application/json'
    }
    
    try:
        # Make the request to get the patent bibliographic data
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        
        # Extract the bibliographic data from the response
        patent_data = response.json()
        if not patent_data:
            raise ValueError("Patent data not found in the response.")
        
        exchange_documents = patent_data.get("ops:world-patent-data", {}).get("exchange-documents", {}).get("exchange-document", [])
        if isinstance(exchange_documents, dict):
            exchange_documents = [exchange_documents]
        first_exchange_document = exchange_documents[0]
        
        # Extract applicants
        applicants = first_exchange_document.get("bibliographic-data", {}).get("parties", {}).get("applicants", {}).get("applicant", [])
        
        if isinstance(applicants, dict):
            applicants = [applicants]

        applicants_name = [applicant.get("applicant-name", {}).get("name", {}).get("$") for applicant in applicants]

        # Extract the contry code (assume that it is the code in brackets given at the end of the first applicant name)
        country_code = applicants_name[0].split()[-1].strip("[]")
            
        # Extract the patent titles
        invention_titles = first_exchange_document.get("bibliographic-data", {}).get("invention-title", [])
        titles = {}
        if invention_titles:
            for title in invention_titles:
                lang = title.get("@lang")
                title_text = title.get("$")
                if lang and title_text:
                    titles[lang] = title_text
                    
        # Extract abstracts
        result_abstracts = first_exchange_document.get("abstract", [])
        if isinstance(result_abstracts, dict):
            result_abstracts = [result_abstracts]
        abstracts = {}
        if result_abstracts:
            for abstract in result_abstracts:
                lang = abstract.get("@lang")
                abstract_text = abstract.get("p", {}).get("$")
                if lang and abstract_text:
                    abstracts[lang] = abstract_text 
        
        
        return {
            "number": number,
            "title": titles,
            "abstract": abstracts,
            "applicants": applicants_name,
            "country": country_code
        }
    
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {e}")
    except KeyError as e:
        raise Exception(f"Unexpected response structure: {e}")
    
patent_biblio = get_patent_biblio(ops_api_url, access_token, type="publication", format="docdb", number="EP4113390A2")
print(f"PatentBiblio: {patent_biblio}")

PatentBiblio: {'number': 'EP4113390A2', 'title': {'de': 'VERFAHREN ZUR DATENVERARBEITUNG, UND ELEKTRONISCHE VORRICHTUNG', 'fr': 'PROCEDE DE TRAITEMENT DES DONNEES, ET DISPOSITIF ELECTRONIQUE', 'en': 'METHOD FOR PROCESSING DATA, AND ELECTRONIC DEVICE'}, 'abstract': {'en': 'The disclosure provides a method for processing data, and an electronic device. The method includes: obtaining first attribute information of input data and second attribute information of a computing device corresponding to the input data; selecting a target operator implementation mode from a plurality of candidate operator implementation modes based on the first attribute information and the second attribute information; determining a plurality of sub-operators included in an operator required for the input data from an operator library based on the target operator implementation mode, to generate the operator; and obtaining an operation result by performing an operation on the input data by the computing device ba

At this time, we have the possibility to get the description and claims of a patent. We can also get the credentials to use the OPS API. Now, we want to get all patents published between two dates.

In [ ]:
def get_patents(api_url: str, consumer_key: str, consumer_secret_key: str, date: str, patent_type: str = "publication") -> list[dict]:
    """Get patents from Ops API based on the given date and type.
    Args:
        api_url (str): The Ops API URL.
        consumer_key (str): The consumer key for authentication.
        consumer_secret_key (str): The consumer secret key for authentication.
        date (str): The date to search for patents (YYYYMMDD).
        patent_type (str): The type of patent to search for (application, priority, publication).
    Returns:
        list[dict]: A list of patents with detailed information.
    """
    
    # Get the access token
    access_token = get_access_token(api_url, consumer_key, consumer_secret_key)
    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Accept': 'application/json'
    }
    
    total_count = 2000
    first_range = 1
    last_range = 100
    patents = []
    
    while total_count > last_range:
        # Construct the URL for the patent search
        url = f"{api_url}/rest-services/published-data/search?Range={first_range}-{last_range}&q=pd=\"{date}\" and pn any \"EP\""
    
        try:
            # Make the request to get the patents matching the criteria
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise an error for bad responses
            
            # Extract the patents from the response
            patent_data = response.json()
            if not patent_data:
                raise ValueError("Patent data not found in the response.")
            
            # Extract the total count of patents
            total_count = int(patent_data.get("ops:world-patent-data", {}).get("ops:biblio-search", {}).get("@total-result-count", 0))
            
            
            print(f"DEBUG: Total count of patents: {total_count}")
            print(f"DEBUG: First range: {first_range}, Last range: {last_range}")
            
            # Update the range for the next request
            total_count = min(total_count, 2000)
            first_range = last_range + 1
            last_range = last_range + 100
            if last_range > total_count:
                last_range = total_count
            
            # Extract publication references
            publications = patent_data.get("ops:world-patent-data", {}).get("ops:biblio-search", {}).get("ops:search-result", {}).get("ops:publication-reference", [])
            
            # Process each publication
            for publication in publications:
                try:
                    document_id = publication.get("document-id", {})
                    doc_number = document_id.get("doc-number", {}).get("$", "")
                    format = document_id.get("@document-id-type", "")
                    kind = document_id.get("kind", {}).get("$", "")
                    country = document_id.get("country", {}).get("$", "")
                    number = f"{country}{doc_number}{kind}"
                    
                    # Fetch detailed data for each patent
                    biblio = get_patent_biblio(api_url, access_token, type=patent_type, format=format, number=number)
                    description = get_patent_description(api_url, access_token, type=patent_type, format=format, number=number)
                    claims = get_patent_claims(api_url, access_token, type=patent_type, format=format, number=number)
                    
                    # Add the patent to the list
                    patents.append({
                        "number": number,
                        "title": biblio.get("title"),
                        "abstract": biblio.get("abstract"),
                        "country": biblio.get("country"),
                        "format": format,
                        "type": patent_type,
                        "publicationDate": f"{date}",
                        "description": description,
                        "claims": claims
                    })
                
                except Exception as e:
                    # Log the error and continue with the next publication
                    print(f"An error occurred while processing patent {number}: {e}")
        
        except Exception as e:
            # Log the error and continue with the next range
            print(f"An error occurred while processing range {first_range}-{last_range}: {e}")
        
    return patents
    
# Example usage
date = "20230104"
patent_type = "publication"
patents = get_patents(ops_api_url, consumer_key, consumer_secret_key, date, patent_type)
print(f"Patents: {patents[:5]}")  # Print the first 5 patents

In [ ]:
print(f"Number of patents found: {len(patents)}")

In [ ]:
# Save the patents to a file
output_file = f"outputs/patents_{date}.json"
with open(output_file, 'w') as f:
    import json
    json.dump(patents, f, indent=4)
print(f"Patents saved to {output_file}")

Patents saved to patents_20230104.json


## Register result in a PostgreSQL database

In this part, we will use the data to create a PostgreSQL database. We will use the psycopg2 package to connect to the database. We will create multiple tables to store the data. The tables will be:
- `patent`: to store the patents
- `patent_claim`: to store the claims
- `patent_description`: to store the description
- `patent_applicant`: to store the applicants

In the `patents` table, we will store the following information:
- `number`: the patent number (PK)
- `en_title`: the title of the patent in English
- `fr_title`: the title of the patent in French
- `de_title`: the title of the patent in German
- `en_abstract`: the abstract of the patent in English
- `fr_abstract`: the abstract of the patent in French
- `de_abstract`: the abstract of the patent in German
- `country`: the country of the patent
- `publication_date`: the publication date of the patent

In the `patent_claim` table, we will store the following information:
- `claim_number`: the id of the claim (PK)
- `patent_number`: the patent number (PK, FK)
- `claim_text`: the text of the claim

In the `patent_description` table, we will store the following information:
- `description_number`: the id of the description (PK)
- `patent_number`: the patent number (PK, FK)
- `description_text`: the text of the description

In the `patent_applicant` table, we will store the following information:
- `applicant_name`: the name of the applicant (PK)
- `patent_number`: the patent number (PK, FK)

### Create the database



In [2]:
db_host = os.environ.get('DB_HOST')
db_port = os.environ.get('DB_PORT')
db_name = os.environ.get('DB_NAME')
db_user = os.environ.get('DB_USER')
db_password = os.environ.get('DB_PASSWORD')

print(f"DBHost: {mask_value(db_host)}")
print(f"DBPort: {mask_value(db_port)}")
print(f"DBName: {mask_value(db_name)}")
print(f"DBUser: {mask_value(db_user)}")
print(f"DBPassword: {mask_value(db_password)}")

DBHost: localhost
DBPort: 5432
DBName: cep
DBUser: user
DBPassword: password


In [4]:
import psycopg2

def get_db_connection():
    """
    Get a database connection using the provided environment variables.
    
    Returns:
        psycopg2.extensions.connection: A connection object to the PostgreSQL database.
        
    Raises:
        Exception: If the connection fails.
    """
    db_host = os.environ.get('DB_HOST')
    db_port = os.environ.get('DB_PORT')
    db_name = os.environ.get('DB_NAME')
    db_user = os.environ.get('DB_USER')
    db_password = os.environ.get('DB_PASSWORD')
    
    try:
        # Establish a connection to the PostgreSQL database
        conn = psycopg2.connect(
            host=db_host,
            port=db_port,
            dbname=db_name,
            user=db_user,
            password=db_password
        )
        return conn
    except Exception as e:
        raise Exception(f"Failed to connect to the database: {e}")
    
conn = get_db_connection()
print("Database connection established.")
    

ModuleNotFoundError: No module named 'psycopg2'

In [29]:
def drop_database_tables():
    """
    Drop the existing tables in the database.
    
    Raises:
        Exception: If the table drop fails.
    """
    try:
        # Create a cursor object to execute SQL commands
        conn = get_db_connection()
        cursor = conn.cursor()
        
        # Drop the patents table if it exists
        cursor.execute("DROP TABLE IF EXISTS patent CASCADE")
        cursor.execute("DROP TABLE IF EXISTS patent_claim")
        cursor.execute("DROP TABLE IF EXISTS patent_description")
        cursor.execute("DROP TABLE IF EXISTS patent_applicant")
        
        # Commit the changes to the database
        conn.commit()

            
    except Exception as e:
        raise Exception(f"Failed to drop database tables: {e}")
    finally:
        cursor.close()
        conn.close()
        
drop_database_tables()

In [30]:
def create_patent_table():
    """
    Create a table for storing patent data in the PostgreSQL database.
    
    Description:
    - `number`: the patent number (PK)
    - `en_title`: the title of the patent in English
    - `fr_title`: the title of the patent in French
    - `de_title`: the title of the patent in German
    - `en_abstract`: the abstract of the patent in English
    - `fr_abstract`: the abstract of the patent in French
    - `de_abstract`: the abstract of the patent in German
    - `country`: the country of the patent
    - `publication_date`: the publication date of the patent
    
    Returns:
        None
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Create the patent table if it doesn't exist
    create_table_query = """
    CREATE TABLE IF NOT EXISTS patent (
        number VARCHAR(255) PRIMARY KEY,
        en_title TEXT,
        fr_title TEXT,
        de_title TEXT,
        en_abstract TEXT,
        fr_abstract TEXT,
        de_abstract TEXT,
        country VARCHAR(10),
        publication_date TEXT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
        

def create_description_table():
    """
    Create a table for storing patent description data in the PostgreSQL database.
    
    Description:
    - `description_number`: the id of the description (PK)
    - `patent_number`: the patent number (PK, FK)
    - `description_text`: the text of the description
    
    Returns:
        None
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Create the description table if it doesn't exist
    create_table_query = """
    CREATE TABLE IF NOT EXISTS patent_description (
        description_number INT,
        patent_number VARCHAR(255) REFERENCES patent(number),
        description_text TEXT,
        PRIMARY KEY (description_number, patent_number)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
        

def create_claim_table():
    """
    Create a table for storing patent claim data in the PostgreSQL database.
    
    Description:
    - `claim_number`: the id of the claim (PK)
    - `patent_number`: the patent number (PK, FK)
    - `claim_text`: the text of the claim
    
    Returns:
        None
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Create the claim table if it doesn't exist
    create_table_query = """
    CREATE TABLE IF NOT EXISTS patent_claim (
        claim_number INT,
        patent_number VARCHAR(255) REFERENCES patent(number),
        claim_text TEXT,
        PRIMARY KEY (claim_number, patent_number)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
    

def create_applicant_table():
    """
    Create a table for storing patent applicant data in the PostgreSQL database.
    
    Description:
    - `applicant_name`: the applicant name (PK)
    - `patent_number`: the patent number (PK, FK)
    
    Returns:
        None
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Create the applicant table if it doesn't exist
    create_table_query = """
    CREATE TABLE IF NOT EXISTS patent_applicant (
        applicant_name TEXT,
        patent_number VARCHAR(255) REFERENCES patent(number),
        PRIMARY KEY (applicant_name, patent_number)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
        

create_patent_table()
create_description_table()
create_claim_table()
create_applicant_table()

In [31]:
def insert_patent_data(patent: dict):
    """
    Insert patent data into the PostgreSQL database.
    
    Args:
        patent (dict): A dictionary containing patent data.
        
    Returns:
        None
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Insert patent data into the patent table
    insert_patent_query = """
    INSERT INTO patent (number, en_title, fr_title, de_title, en_abstract, fr_abstract, de_abstract, country, publication_date)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (number) DO NOTHING;
    """
    
    # Extract the title and abstract in different languages
    title = patent.get("title", {})
    abstract = patent.get("abstract", {})
    
    cursor.execute(insert_patent_query, (
        patent["number"],
        title.get("en") if title.get("en") else None,
        title.get("fr") if title.get("fr") else None,
        title.get("de") if title.get("de") else None,
        abstract.get("en") if abstract.get("en") else None,
        abstract.get("fr") if abstract.get("fr") else None,
        abstract.get("de") if abstract.get("de") else None,
        patent["country"],
        patent["publicationDate"]
    ))
    
    # Insert claims into the patent_claim table
    for claim in patent["claims"]:
        insert_claim_query = """
        INSERT INTO patent_claim (claim_number, patent_number, claim_text)
        VALUES (%s, %s, %s)
        ON CONFLICT (claim_number, patent_number) DO NOTHING;
        """
        
        # Claim example: "1. A method for processing..."
        
        # Extract the claim number and text
        claim_number = claim.split(".")[0].strip()
        claim_text = claim[len(claim_number)+1:].strip()  # Skip the number (one or two digits) and the dot
        
        if not claim_number or not claim_text:
            raise ValueError(f"Invalid claim format: {claim[:20]}...")

        cursor.execute(insert_claim_query, (
            int(claim_number),
            patent["number"],
            claim_text
        ))
    
    # Insert description into the patent_description table
    for description in patent["description"]:
        insert_description_query = """
        INSERT INTO patent_description (description_number, patent_number, description_text)
        VALUES (%s, %s, %s)
        ON CONFLICT (description_number, patent_number) DO NOTHING;
        """
        
        # Description example: "TECHNICAL FIELD",
        # Description example: "[0001]    The disclosure relates to the..."
        
        # Extract the claim number and text. We want only descriptions starting with [xxxx].
        
        if not description.startswith("["):
            continue
        
        description_number = description[1:5].strip()  # Skip the [ and the last ]
        description_text = description[6:].strip()
        
        if not description_number or not description_text:
            raise ValueError(f"Invalid description format: {description[:20]}...")
        
        cursor.execute(insert_description_query, (
            int(description_number),
            patent["number"],
            description_text
        ))
        
    # Insert applicants into the patent_applicant table
    for applicant in patent["applicants"]:
        insert_applicant_query = """
        INSERT INTO patent_applicant (applicant_name, patent_number)
        VALUES (%s, %s)
        ON CONFLICT (applicant_name, patent_number) DO NOTHING;
        """
        
        cursor.execute(insert_applicant_query, (
            applicant,
            patent["number"]
        ))
    
    conn.commit()
    cursor.close()
    conn.close()
        

In [18]:
# Test
patent = {
    "number": "EP0000000",
    "title": {
        "de": "Beispiel Titel",
        "fr": "Brevet test",
        "en": "Test patent"
    },
    "abstract": {
        "de": "Dies ist ein Beispiel-Abstract",
        "fr": "Ce est un exemple d'abstract",
        "en": "This is an example abstract"
    },
    "country": "FR",
    "format": "docdb",
    "type": "publication",
    "publicationDate": "20230104",
    "description": [
        "TECHNICAL FIELD",
        "[0001]    The disclosure relates ...",
        "BACKGROUND",
        "[0002]    Deep learning framework ...",
        "[0003]    However, the deep learning framework ...",
        "SUMMARY",
    ],
    "claims": [
        "1. A method for processing...",
        "2. A system for processing...",
        "3. A device for processing..."
    ],
    "applicants": [
        "Applicant 1",
        "Applicant 2"
    ]
}

insert_patent_data(patent)
print(f"Patent data inserted for {patent['number']}.")

Patent data inserted for EP0000000.


In [19]:
def get_patent(number: str):
    """
    Get patent data from the PostgreSQL database.
    
    Args:
        number (str): The patent number.
        
    Returns:
        dict: A dictionary containing patent data.
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Fetch the patent data from the database
    fetch_patent_query = """
    SELECT * FROM patent
    WHERE patent.number = %s;
    """
    
    cursor.execute(fetch_patent_query, (number,))
    result = cursor.fetchone()
    
    patent = {
        "number": result[0],
        "en_title": result[1],
        "fr_title": result[2],
        "de_title": result[3],
        "en_abstract": result[4],
        "fr_abstract": result[5],
        "de_abstract": result[6],
        "country": result[7],
        "publication_date": result[8],
        "description": [],
        "claims": [],
        "applicants": []
    }
    
    cursor.close()
    
    # Fetch the claims from the database
    fetch_claims_query = """
    SELECT claim_number, claim_text
    FROM patent_claim
    WHERE patent_number = %s;
    """
    cursor = conn.cursor()
    cursor.execute(fetch_claims_query, (number,))
    claims = cursor.fetchall()
    for claim in claims:
        patent["claims"].append({
            "claim_number": claim[0],
            "claim_text": claim[1]
        })
    cursor.close()
    
    # Fetch the description from the database
    fetch_description_query = """
    SELECT description_number, description_text
    FROM patent_description
    WHERE patent_number = %s;
    """
    cursor = conn.cursor()
    cursor.execute(fetch_description_query, (number,))
    descriptions = cursor.fetchall()
    for description in descriptions:
        patent["description"].append({
            "description_number": description[0],
            "description_text": description[1]
        })
    cursor.close()
    
    # Fetch the applicants from the database
    fetch_applicants_query = """
    SELECT applicant_name
    FROM patent_applicant
    WHERE patent_number = %s;
    """
    cursor = conn.cursor()
    cursor.execute(fetch_applicants_query, (number,))
    applicants = cursor.fetchall()
    for applicant in applicants:
        patent["applicants"].append(applicant[0])
    cursor.close()
    
    # Close the database connection
    conn.close()
    
    return patent

# Test
patent_number = "EP0000000"
patent_data = get_patent(patent_number)

import pprint
pprint.pprint(patent_data)

{'applicants': ['Applicant 1', 'Applicant 2'],
 'claims': [{'claim_number': 1, 'claim_text': 'A method for processing...'},
            {'claim_number': 2, 'claim_text': 'A system for processing...'},
            {'claim_number': 3, 'claim_text': 'A device for processing...'}],
 'country': 'FR',
 'de_abstract': 'Dies ist ein Beispiel-Abstract',
 'de_title': 'Beispiel Titel',
 'description': [{'description_number': 1,
                  'description_text': 'The disclosure relates ...'},
                 {'description_number': 2,
                  'description_text': 'Deep learning framework ...'},
                 {'description_number': 3,
                  'description_text': 'However, the deep learning framework '
                                      '...'}],
 'en_abstract': 'This is an example abstract',
 'en_title': 'Test patent',
 'fr_abstract': "Ce est un exemple d'abstract",
 'fr_title': 'Brevet test',
 'number': 'EP0000000',
 'publication_date': '20230104'}


## Final step

In this last part, we will use the OPS API to retrieve the data for multiple dates. We will modify the `get_patents` function to get the patents for multiple dates and store the data in the PostgreSQL database.

In [ ]:
import time

def get_patents(api_url: str, consumer_key: str, consumer_secret_key: str, dates: list[str], patent_type: str = "publication"):
    """Get patents from Ops API based on the given dates and type and save them into the database.
    Args:
        api_url (str): The Ops API URL.
        consumer_key (str): The consumer key for authentication.
        consumer_secret_key (str): The consumer secret key for authentication.
        dates (list[str]): The dates to search for patents (YYYYMMDD).
        patent_type (str): The type of patent to search for (application, priority, publication).
    Returns:
        None
    Raises:
        Exception: If the request fails or the patent data is not found.
    """
    for date in dates:
        print(f"Processing patents for date: {date}")

        # Get the access token
        access_token = get_access_token(api_url, consumer_key, consumer_secret_key)

        headers = {
            'Authorization': f'Bearer {access_token}',
            'Accept': 'application/json'
        }

        total_count = 2000
        first_range = 1
        last_range = 100

        while total_count > last_range:
            # Construct the URL for the patent search
            url = f"{api_url}/rest-services/published-data/search?Range={first_range}-{last_range}&q=pd=\"{date}\" and pn any \"EP\""

            try:
                # Make the request to get the patents matching the criteria
                response = requests.get(url, headers=headers)
                response.raise_for_status()  # Raise an error for bad responses

                # Extract the patents from the response
                patent_data = response.json()
                if not patent_data:
                    raise ValueError("Patent data not found in the response.")

                # Extract the total count of patents
                total_count = int(patent_data.get("ops:world-patent-data", {}).get("ops:biblio-search", {}).get("@total-result-count", 0))

                # Update the range for the next request
                total_count = min(total_count, 2000)
                first_range = last_range + 1
                last_range = last_range + 100
                if last_range > total_count:
                    last_range = total_count

                # Extract publication references
                publications = patent_data.get("ops:world-patent-data", {}).get("ops:biblio-search", {}).get("ops:search-result", {}).get("ops:publication-reference", [])

                # Process each publication
                for publication in publications:
                    try:
                        document_id = publication.get("document-id", {})
                        doc_number = document_id.get("doc-number", {}).get("$", "")
                        format = document_id.get("@document-id-type", "")
                        kind = document_id.get("kind", {}).get("$", "")
                        country = document_id.get("country", {}).get("$", "")
                        number = f"{country}{doc_number}{kind}"

                        # Fetch detailed data for each patent
                        biblio = get_patent_biblio(api_url, access_token, type=patent_type, format=format, number=number)
                        description = get_patent_description(api_url, access_token, type=patent_type, format=format, number=number)
                        claims = get_patent_claims(api_url, access_token, type=patent_type, format=format, number=number)

                        # Add the patent to the list
                        patent = {
                            "number": number,
                            "title": biblio.get("title"),
                            "abstract": biblio.get("abstract"),
                            "country": biblio.get("country"),
                            "applicants": biblio.get("applicants"),
                            "format": format,
                            "type": patent_type,
                            "publicationDate": f"{date}",
                            "description": description,
                            "claims": claims
                        }

                        # Insert the patent data into the database
                        insert_patent_data(patent)

                    except requests.exceptions.HTTPError as e:
                        if e.response.status_code == 403:
                            rejection_reason = e.response.headers.get("X-Rejection-Reason", "Unknown")
                            if rejection_reason == "IndividualQuotaPerHour":
                                print("Hourly quota exhausted. Sleeping for 1 hour and 5 minutes...")
                                time.sleep(3900)  # Sleep for 1 hour 5 minutes
                                access_token = get_access_token(api_url, consumer_key, consumer_secret_key)  # Refresh token
                                headers['Authorization'] = f'Bearer {access_token}'  # Update headers
                                continue
                            else:
                                raise Exception(f"403 Forbidden error due to quota: {rejection_reason}. Exiting.")
                        else:
                            raise Exception(f"Request failed: {e}")

                    except Exception as e:
                        # Log the error and continue with the next publication
                        print(f"An error occurred while processing patent {number}: {e}")

            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 403:
                    rejection_reason = response.headers.get("X-Rejection-Reason", "Unknown")
                    if rejection_reason == "IndividualQuotaPerHour":
                        print("Hourly quota exhausted. Sleeping for 1 hour and 5 minutes...")
                        time.sleep(3900)  # Sleep for 1 hour 5 minutes
                        access_token = get_access_token(api_url, consumer_key, consumer_secret_key)  # Refresh token
                        headers['Authorization'] = f'Bearer {access_token}'  # Update headers
                        continue
                    else:
                        raise Exception(f"403 Forbidden error due to quota: {rejection_reason}. Exiting.")
                else:
                    raise Exception(f"Request failed: {e}")

            except Exception as e:
                # Log the error and continue with the next range
                print(f"An error occurred while processing range {first_range}-{last_range}: {e}")
            
dates = [
    # "20250305",
    # "20241204",
    #"20240904",
    #"20240605",
    #"20240306",
    #"20231206",
    # "20230906",
    # "20230607", 
    # "20230301", 
    # "20221207", 
    # "20220907", 
    # "20220601", 
    # "20220302", 
    # "20211201", 
    # "20210901", 
    # "20210602", 
    # "20210303"
]
patent_type = "publication"
get_patents(ops_api_url, consumer_key, consumer_secret_key, dates, patent_type)
print("Patents data inserted into the database.")

Processing patents for date: 20231206
An error occurred while processing patent EP4287527A2: invalid literal for int() with base 10: 'R(ψI'
An error occurred while processing patent EP4287104A2: invalid literal for int() with base 10: '1. 2'
An error occurred while processing patent EP4285908A2: invalid literal for int() with base 10: '3] T'
An error occurred while processing patent EP4287245A2: Invalid claim format: here, L1 is selected...
An error occurred while processing patent EP4286012A2: Invalid claim format: or a salt or isomer ...
An error occurred while processing patent EP4286491A2: Invalid claim format: wherein in Formula 1...
An error occurred while processing patent EP4272825A2: Invalid claim format: or a pharmaceuticall...
An error occurred while processing patent EP4286001A2: Invalid claim format: wherein R11 is C1-C6...
An error occurred while processing patent EP4285723A2: Invalid claim format: wherein
R<1> and R<2...
An error occurred while processing patent EP428651